
## Why ODC for DEA?

-  Shift in paradigm from scene based analysis to pixel based.
-  A simple, standard, and extensible way to interact with and perform analysis on data.
-  Allow users to define exactly in what state data is to be delivered in terms of projection, resolution, bands and datasets.
-  Easily create subsets or derivative products that are managed and tracked by the ODC.
-  Provide a base set of functions and tools that make analysis more simple!


### Let's get right into it by loading a cube.



In [ ]:
%%time
%matplotlib notebook
import datacube

dc = datacube.Datacube(app='load-data-example')

# Let's request a datacube!!
# By default, the x and y arguments accept queries in a geographical co-ordinate system WGS84, 
# identified by the EPSG code 4326.

data_cube = dc.load(
    product='s2a_nrt_granule',
    x=(146.30, 146.40), y=(-43.30,-43.20),
    #time=('2018-09-01', '2018-10-01'),
    resolution = (-10,10),
    output_crs='epsg:3577',
    measurements=('nbar_red','nbar_green','nbar_blue','nbar_nir_1','fmask')
)


In [ ]:
print(data_cube)

## Let's explore what a data cube is.

In the context of earth observation a data cube is a multi-dimensional array that will consistantly contain the dimensions of time and a spatial grid. Along these dimensions any number of measurements can be associated to these grids and will represent the measurement taken at a specific time and space. 

<img src="images/Data_Cube.png">

The ODC uses the XArray library http://xarray.pydata.org/en/stable/, the key to building effective ODC algorithms is to master the use of the xarray. Please click on the documention to find out more otherwise a number of examples are provided below.

They're also python libraries such as Dask http://dask.pydata.org/en/latest/ that can scale up computation of XArray to supercomputing, and cloud solutions. They're many other project utilising XArray and it becoming a standard for multi-dimenional data analysis.


In [ ]:
%matplotlib notebook
# Pixel Drill by index through time
pixel_drill_index = data_cube.isel(x=[500], y=[1000])
print(pixel_drill_index)
print('----------------------------------------------')

# Pixel Drill by coordinates through time.
pixel_drill_coordinates = data_cube.sel(x=[-4788000], y=[1197000], method='nearest')
print(pixel_drill_coordinates)
print('----------------------------------------------')

# Pixel Drill by coordinates through time.
pixel_slice_index = data_cube.isel(x=slice(200,300), y=slice(200,300), time=2)
print(pixel_slice_index)


## Querying a data cube.

Our datacube can now be sliced, trimmed or querried with masks, you can basically sculpt any object out of the cube pixels.

<img src="images/pixel_drill.png">
<img src="images/line_drill.png">


In [ ]:
pixel_drill_index.nbar_nir_1.plot()
#pixel_drill_index.nbar_red.plot()
#pixel_drill_index.nbar_green.plot()
#pixel_drill_index.nbar_blue.plot()

In [ ]:
%matplotlib notebook
# Operations on cubes.
monthly_means = pixel_drill_coordinates.groupby('time.month').mean()
print(monthly_means)
print('---------------------------------')

y_cross_sections = pixel_slice_index.groupby('x').mean()
print(y_cross_sections)
y_cross_sections.nbar_nir_1.plot()

## Masking using DEA provided meta-data.

Next we'll explore sentinel 2 fmask, each pixel in the QA bands of products contain integers or bit packed combinations that represent surface, atmospheric, and sensor conditions that can affect the overall usefulness of a given pixel.

Used effectively, these masks improve the integrity of science investigations by indicating which pixels might be affected by instrument artifacts or subject to cloud contamination. The ODC has in built methods that transform the metadata of a specific product to a libary of masking templates. For example, you are able to build a cloud mask directly from the ODC library.

In [ ]:
from datacube.storage import masking  # Import masking capabilities
print(masking.describe_variable_flags(data_cube)) #Describe the masks available


In [ ]:
clean_pixel_mask = masking.make_mask(
    data_cube.fmask,
    fmask = 'valid')

print(clean_pixel_mask)

masked_cloud = data_cube.where(clean_pixel_mask)
print (masked_cloud)


In [ ]:
%matplotlib notebook
from utils.utils import three_band_image
import matplotlib.pyplot as plt

#img_toshowNoMask = three_band_image(masked_cloud,bands = ['nbar_red', 'nbar_green', 'nbar_blue'], time = 1)

time = 9
img_toshowNoMask = three_band_image(masked_cloud,bands = ['nbar_red', 'nbar_green', 'nbar_blue'], tim =time)
img_toshow = three_band_image(data_cube,bands = ['nbar_red', 'nbar_green', 'nbar_blue'], time=time)
#fig, (ax1, ax2) = plt.subplots(2, 1)
fig = plt.figure(1, [10,20])

plt.subplot(211)
ax = plt.gca()
ax.set_title("All Pixels", fontweight='bold', fontsize=16)
ax.set_xticklabels(data_cube.x.values)
ax.set_yticklabels(data_cube.y.values)
ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')

plt.imshow(img_toshow)

plt.subplot(212)
ax = plt.gca()
ax.set_title("Artifacts Filtered Out", fontweight='bold', fontsize=16)
ax.set_xticklabels(data_cube.x.values)
ax.set_yticklabels(data_cube.y.values)
ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')
plt.imshow(img_toshowNoMask)


## Merging two datacube into one.

In this example we'll load a datacube over the same time period and extent for sentinal 2a sister satellite 2b, we'll then merge them into a single cube.


In [ ]:
import xarray

data_cube_sister = dc.load(
    product='s2b_nrt_granule', 
    x=(146.30, 146.40), y=(-43.30,-43.20),
    #time=('2018-09-01', '2018-10-01'),
    resolution = (-10,10),
    output_crs='epsg:3577',
    measurements=('nbar_red','nbar_green','nbar_blue','nbar_nir_1','fmask')
)

crs = data_cube_sister.crs
affine = data_cube_sister.affine

combined_cube = xarray.merge([data_cube,data_cube_sister])
combined_cube.attrs['crs'] = crs
combined_cube.attrs['affine'] = affine


In [ ]:
combined_cube